In [1]:
import pyarrow as pa
from pyarrow import csv
from datasets import load_from_disk, Dataset, DatasetDict
import transformers 
from transformers import Trainer
import pytorch_lightning as pl
import torch
import pandas as pd
import matplotlib.pyplot as plt

2023-01-04 02:46:47.955305: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 02:46:48.118741: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 02:46:48.118788: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-04 02:46:49.053522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
train_datasets = load_from_disk('/opt/ml/input/data/train_dataset/data_wiki_korquad_ver2/')

In [3]:
dataset = pd.DataFrame(columns=train_datasets['train'].column_names)
for f in train_datasets['train'].column_names:
    dataset[f] = train_datasets['train'][f]
dataset

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293.0,42.0
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638.0,2873.0
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028.0,230.0
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146.0,992.0
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334.0,548.0
...,...,...,...,...,...,...,...
125430,뉴델리_메탈로-베타락타마제,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,6467478-1-1,"{'answer_start': [73], 'text': ['인도']}",NaN,NaN
125431,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,6467478-2-0,"{'answer_start': [253], 'text': ['유전자 전달']}",NaN,NaN
125432,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,6467478-2-1,"{'answer_start': [129], 'text': ['37건']}",NaN,NaN
125433,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,6490801-2-0,"{'answer_start': [404], 'text': ['콜리스틴']}",NaN,NaN


In [4]:
valid_data = train_datasets['validation']

In [5]:
index = dataset.context.str.len().sort_values().index


In [6]:
total_data = dataset.reindex(index)
total_data.drop('__index_level_0__', axis=1, inplace=True)

In [7]:

total_datasets = Dataset.from_pandas(total_data)
total_datasets = DatasetDict({'train': total_datasets, 'validation': valid_data})
total_datasets.save_to_disk('/opt/ml/input/data/train_dataset/cur_korquad')

In [9]:
easy_data = dataset.reindex(index).iloc[:1976, :]
hard_data = dataset.reindex(index).iloc[1976:, :]

In [16]:
easy_data.drop('__index_level_0__', axis=1, inplace=True)
hard_data.drop('__index_level_0__', axis=1, inplace=True)

In [18]:
easy_dataset = Dataset.from_pandas(easy_data)
hard_dataset = Dataset.from_pandas(hard_data)

In [21]:
easy_datasets = DatasetDict({'train': easy_dataset, 'validation': valid_data})
hard_datasets = DatasetDict({'train': hard_dataset, 'validation': valid_data})

In [22]:
easy_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 1976
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [23]:
hard_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 1976
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [24]:
easy_datasets.save_to_disk('/opt/ml/input/data/train_dataset/easy_data')
easy_datasets.save_to_disk('/opt/ml/input/data/train_dataset/hard_data')

In [29]:
data = load_from_disk('/opt/ml/input/data/train_dataset/easy_data/')
data

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 1976
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})